In [156]:
import time


from collections import Counter


Mycket av koden är hämtat från https://github.com/udacity/deep-learning/tree/master/embeddings 

In [77]:
PATH = 'data/'

In [78]:
%%time
with open(f'{PATH}dec1980.txt') as f:
    text = f.read()

CPU times: user 767 ms, sys: 543 ms, total: 1.31 s
Wall time: 1.32 s


In [79]:
# Begränsa textstorleken
text = text[:5000000]

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [80]:
def preprocess(text):

    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', ' <COLON> ')
    words = text.split()
    
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]

    return trimmed_words

In [81]:
# Detta tar en stund
# %%time #CPU times: user 48.3 s, sys: 7.27 s, total: 55.6 s
words = preprocess(text)
print(words[:30])

['ur', 'kb', '<COLON>', 's', 'samlingar', 'digitaliserad', 'år', 'statens', 'offentliga', 'utredningar', '1981', '<COLON>', '68', 'svenska', 'kyrkans', 'gudstjänst', 'kyrkliga', 'bilaga', 'handlingar', '5', 'döden', 'döendet', '<COMMA>', 'och', 'begravningsgudstjänsten', 'av', '1968', 'års', 'stockholm', '1981']


In [82]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 696685
Unique words: 9688
CPU times: user 35.3 ms, sys: 987 µs, total: 36.3 ms
Wall time: 36.2 ms


And here I'm creating dictionaries to covert words to integers and backwards, integers to words. The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1 and so on. The words are converted to integers and stored in the list `int_words`.

In [83]:
def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict....
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab, word_counts

In [84]:
vocab_to_int, int_to_vocab, word_counts = create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [85]:
word_counts.most_common()

[('<PERIOD>', 54345),
 ('i', 20362),
 ('<COMMA>', 18173),
 ('att', 16004),
 ('och', 15762),
 ('av', 14466),
 ('som', 11890),
 ('för', 10943),
 ('en', 10711),
 ('det', 9171),
 ('den', 8311),
 ('de', 7995),
 ('till', 7343),
 ('är', 7169),
 ('med', 7145),
 ('på', 6845),
 ('<COLON>', 5805),
 ('har', 5637),
 ('om', 5338),
 ('ett', 5217),
 ('<RIGHT_PAREN>', 4343),
 ('kan', 4234),
 ('<LEFT_PAREN>', 4202),
 ('inte', 3937),
 ('eller', 2696),
 ('vi', 2472),
 ('detta', 2404),
 ('sou', 2304),
 ('skall', 2079),
 ('från', 2076),
 ('2', 2053),
 ('vid', 2033),
 ('man', 2025),
 ('bör', 2013),
 ('1', 1957),
 ('-', 1906),
 ('under', 1857),
 ('också', 1829),
 ('inom', 1822),
 ('3', 1818),
 ('m', 1698),
 ('olika', 1617),
 ('vara', 1579),
 ('sig', 1570),
 ('a', 1563),
 ('dessa', 1526),
 ('5', 1457),
 ('andra', 1445),
 ('än', 1416),
 ('denna', 1377),
 ('så', 1353),
 ('genom', 1340),
 ('s', 1301),
 ('mellan', 1299),
 ('även', 1268),
 ('skulle', 1199),
 ('år', 1192),
 ('4', 1155),
 ('finns', 1096),
 ('gäller',

In [86]:
int_words[:20]

[267,
 7650,
 16,
 52,
 7651,
 7652,
 56,
 331,
 760,
 1462,
 70,
 16,
 603,
 107,
 1676,
 3052,
 3053,
 193,
 3588,
 46]

In [88]:
npint=np.array(int_words)

In [89]:
def konk_rad(index, length):
    w=np.where(npint==index)[0]
    print (int_to_vocab[index])
    for item in w:
        r=np.arange(item-length, item+length)
        k=npint[r]
        print ([int_to_vocab[word] for word in k])

In [90]:
konk_rad(21,8)

kan
['har', 'döden', 'satts', 'på', 'undantag', '-', 'nog', '<COMMA>', 'kan', 'man', 'med', 'dess', '<COMMA>', '<COMMA>', 'de', '<PERIOD>']
['denna', 'inställning', 'ha', 'börjat', 'att', 'förändras', '<QUESTION_MARK>', 'man', 'kan', 'denna', 'till', 'bok', 'on', 'death', 'and', 'dying']
['gör', 'den', 'lämnar', 'den', 'mänskliga', '<COMMA>', 'lika', 'litet', 'kan', 'man', 'säga', '<COMMA>', 'att', 'livet', 'där', 'livet']
['både', 'och', 'död', 'är', 'från', 'ett', 'övergången', 'en', 'kan', 'det', 'anses', 'vara', 'öppen', 'fråga', '<COMMA>', 'om']
['är', 'en', 'integrerad', 'del', 'av', 'den', 'mänskliga', 'och', 'kan', 'vara', 'en', 'döden', 'skulle', 'det', 'knappast', 'ha']
['och', 'skapande', 'borde', 'från', 'kristen', 'utgångspunkt', 'främmande', '<PERIOD>', 'kan', 'ses', 'som', 'ett', 'av', 'de', 'mest', 'döden']
['synen', 'på', 'inte', 'döden', 'men', 'å', 'andra', 'sidan', 'kan', 'den', 'kristna', 'döden', 'sedan', 'ett', 'isolerat', 'fenomen']
['<LEFT_PAREN>', '<RIGHT_PARE

['att', '<PERIOD>', 'innebär', 'varje', 'ella', 'som', 'sammanlagda', 'detta', 'kan', 'dessa', 'den', 'genom', 'uppkomma', '<COMMA>', 'production', 'serie']
['2', '<PERIOD>', '5', '<RIGHT_PAREN>', '<PERIOD>', 'denna', 'kommer', 'c', 'kan', 'sätt', '<COLON>', 'att', 'jämföras', 'självkostnaden', 'med', 'för']
['situationer', '<COMMA>', 'för', 'vad', 'det', 'det', 'inte', 'skulle', 'kan', 'som', 'händer', 'av', 'att', 'räknar', 'dessutom', 'skulle']
['kan', 'som', 'händer', 'av', 'att', 'räknar', 'dessutom', 'skulle', 'kan', 'vara', '<PERIOD>', 'svårt', '<PERIOD>', 'det', 'att', 'ju']
['som', 'fmv', 'annars', 'av', 'undvika', 'givetvis', '<COMMA>', 'fråga', 'kan', 'denna', 'klargöra', '<COMMA>', 'som', 'upprätthållande', 'efter', 'b']
['i', '<PERIOD>', 'annat', 'skulle', 'kunde', 'vapensystem', 'ett', 'annan', 'kan', 'med', 'nästa', 'om', 'när', 'fmv', 'reducera', 'vara']
['kategorin', 'nämnda', '<PERIOD>', 'utveckla', 'der', '<PERIOD>', 'tillverka', 'emellertid', 'kan', 'måste', 'tillsa

['vissa', 'alltså', 'våld', '<COMMA>', 'heller', 'att', 'inte', 'han', 'kan', 'henne', '<PERIOD>', 'enkel', 'invändningar', 'men', 'det', 'kvinnan']
['inte', 'åt', 'mat', 'med', 'han', 'är', 'han', 'säger', 'kan', 'och', 'mat', 'av', 'fästmannen', 'hon', 'är', 'i']
['hon', 'är', 'i', 'en', '<PERIOD>', 'vila', '<PERIOD>', 'upp', 'kan', 'på', 'eftermiddagen', 'hennes', 'ålder', 'en', 's', 'k']
['den', 'andra', 'gäller', 'mot', 'sig', 'udda', 'att', 'man', 'kan', 'våld', 'de', 'sexuella', '<LEFT_PAREN>', '4', '<RIGHT_PAREN>', 'här']
['sam-', 'själva', 'våldtäkt', '<COMMA>', 'till', 'verkar', 'på', 'det', 'kan', 'man', 'belägga', 'från', 'faktorer', '<COLON>', 'allt-', 'hon']
['olika', 'till', 'om', 'en', 'massa', 'moment', 'ansvarig', 'att', 'kan', 'det', 'förhållande', 'fråga', 'gärningen', 'som', 'är', 'samlag']
['redan', 'som', 'är', 'och', 'berusad', '<COMMA>', 'den', 'i', 'kan', 'göras', 'eller', 'det', 'till', 'som', 'anges', 'dessa']
['våld', 'med', 'eller', 'framstår', 'som', 'en'

In [91]:
def print_vocab (index, length):
    for i in range (length):
        print (int_to_vocab[index+i])

In [92]:
print_vocab(1000, 10)

ligga
jord
varierar
31
väljer
kortare
ändring
övergripande
målen
fastighet


In [93]:
def make_pd(words):
    c = Counter(words)
    n1 = np.array(list(c.items()))
    return pd.DataFrame(n1, columns=['word', 'count'])

In [139]:
c1=Counter(words); c1

Counter({'ur': 292,
         'kb': 7,
         '<COLON>': 5805,
         's': 1301,
         'samlingar': 7,
         'digitaliserad': 7,
         'år': 1192,
         'statens': 236,
         'offentliga': 107,
         'utredningar': 54,
         '1981': 930,
         '68': 131,
         'svenska': 678,
         'kyrkans': 46,
         'gudstjänst': 23,
         'kyrkliga': 23,
         'bilaga': 371,
         'handlingar': 19,
         '5': 1457,
         'döden': 89,
         'döendet': 8,
         '<COMMA>': 18173,
         'och': 15762,
         'begravningsgudstjänsten': 10,
         'av': 14466,
         '1968': 58,
         'års': 319,
         'stockholm': 103,
         'lars': 23,
         'ab': 97,
         'isbn': 9,
         'issn': 9,
         '0375-250x': 6,
         'sou': 2304,
         'innehåll': 150,
         '<PERIOD>': 54345,
         '1': 1957,
         'begravningens': 16,
         '3': 1818,
         'teologiska': 7,
         'bakgrund': 192,
         'den': 8

In [95]:
n1 = np.array(list(c1.items()))

In [97]:
p1=pd.DataFrame(n1); p1.tail()

,0,1
9683,3c,6
9684,målsäganden,7
9685,fio,6
9686,angive1se,8
9687,tidsrymnden,7


In [98]:
w2=words

In [99]:
p2 = make_pd(w2)

In [100]:
p2['count'] = p2['count'].astype(int)

In [116]:
p2.dtypes

word      object
count      int64
div      float64
dtype: object

In [124]:
p2['count'].sum()

696685

In [103]:
p2['div']=p2['count']/p2['count'].sum()

In [125]:
p2.head()

,word,count,div
0,ur,292,0.000419
1,kb,7,0.000010
2,<COLON>,5805,0.008332
3,s,1301,0.001867
4,samlingar,7,0.000010


In [108]:
def make_table(words):
    df = make_pd(words)
    df['count'] = df['count'].astype(int)
    df['div']=df['count']/df['count'].sum()
    return df

In [109]:
p1 = make_table(words[20000:])

In [110]:
p1.head()

,word,count,div
0,fall,672,0.000993
1,vid,1952,0.002885
2,större,720,0.001064
3,<PERIOD>,52810,0.078042
4,det,8926,0.013191


In [123]:
p1.loc[p1['word']=='större']['div']

2    0.001064
Name: div, dtype: float64

In [119]:
for word in p1['word']:
    print (abs(math.log(p1.loc[p1['word']==word]['div']/p2.loc[p2['word']==word]['div'])))
    

TypeError: cannot convert the series to <class 'float'>

In [135]:
n1=np.array(p1)
n2=np.array(p2)

In [134]:
n1[:,0]

'fall'

In [138]:
n2[0](n1[:,0])

TypeError: 'numpy.ndarray' object is not callable

In [149]:
def make_freq (words):
    counter = Counter(words)
    sum_of = sum(counter.values())
    return {x:(counter[x]/sum_of) for x in counter}

In [142]:
s = sum(c1.values())

In [143]:
f1={x:(c1[x]/s) for x in c1}

In [144]:
f1

{'ur': 0.00041912772630385326,
 'kb': 1.0047582479886893e-05,
 '<COLON>': 0.00833231661367763,
 's': 0.001867414972333264,
 'samlingar': 1.0047582479886893e-05,
 'digitaliserad': 1.0047582479886893e-05,
 'år': 0.0017109597594321682,
 'statens': 0.0003387470664647581,
 'offentliga': 0.00015358447504969965,
 'utredningar': 7.75099219876989e-05,
 '1981': 0.0013348931008992585,
 '68': 0.0001880333292664547,
 'svenska': 0.0009731801316233305,
 'kyrkans': 6.602697058211387e-05,
 'gudstjänst': 3.301348529105694e-05,
 'kyrkliga': 3.301348529105694e-05,
 'bilaga': 0.0005325218714340054,
 'handlingar': 2.7272009588264423e-05,
 '5': 0.002091332524742172,
 'döden': 0.00012774783438713336,
 'döendet': 1.1482951405585021e-05,
 '<COMMA>': 0.02608495948671207,
 'och': 0.022624285006853886,
 'begravningsgudstjänsten': 1.4353689256981276e-05,
 'av': 0.020764046879149112,
 '1968': 8.32513976904914e-05,
 'års': 0.0004578826872977027,
 'stockholm': 0.00014784299934690715,
 'lars': 3.301348529105694e-05,
 '

In [145]:
sum(f1.values())

1.000000000000025

In [146]:
s

696685

In [ ]:
dict2 = {x:dict1[x] for x in keys}

In [147]:
words_sub = words[:30000]; len(words_sub)

30000

In [150]:
freq_sub = make_freq(words_sub)

In [151]:
freq_sub

{'ur': 0.0017333333333333333,
 'kb': 3.3333333333333335e-05,
 '<COLON>': 0.0169,
 's': 0.004566666666666667,
 'samlingar': 3.3333333333333335e-05,
 'digitaliserad': 3.3333333333333335e-05,
 'år': 0.0002666666666666667,
 'statens': 3.3333333333333335e-05,
 'offentliga': 0.0001,
 'utredningar': 3.3333333333333335e-05,
 '1981': 0.002533333333333333,
 '68': 0.002533333333333333,
 'svenska': 0.0008333333333333334,
 'kyrkans': 0.0009333333333333333,
 'gudstjänst': 0.00046666666666666666,
 'kyrkliga': 0.0002666666666666667,
 'bilaga': 3.3333333333333335e-05,
 'handlingar': 3.3333333333333335e-05,
 '5': 0.002,
 'döden': 0.0024,
 'döendet': 0.00013333333333333334,
 '<COMMA>': 0.055433333333333334,
 'och': 0.0297,
 'begravningsgudstjänsten': 6.666666666666667e-05,
 'av': 0.013733333333333334,
 '1968': 0.0004,
 'års': 0.0005,
 'stockholm': 0.0001,
 'lars': 3.3333333333333335e-05,
 'ab': 3.3333333333333335e-05,
 'isbn': 3.3333333333333335e-05,
 'issn': 3.3333333333333335e-05,
 '0375-250x': 3.33333

In [152]:
def compare_freqs (sub_dict, total_dict):
    return {word:(sub_dict[word]/total_dict[word]) for word in sub_dict}
    

In [158]:
%%time

representation = compare_freqs (freq_sub, f1); representation


CPU times: user 568 µs, sys: 90 µs, total: 658 µs
Wall time: 663 µs


In [165]:
list1 = sorted(representation, key=representation.get, reverse=True)

sorted_compared = {word:representation[word] for word in list1}
sorted_compared

{'teologiska': 23.222833333333334,
 'begravningsritualets': 23.222833333333334,
 'dopet': 23.222833333333334,
 'gravprocessionen': 23.222833333333334,
 'griftetalet': 23.222833333333334,
 'varda': 23.222833333333334,
 'kommen': 23.222833333333334,
 'uppväcka': 23.222833333333334,
 'formel': 23.222833333333334,
 'formeln': 23.222833333333334,
 'valedictio': 23.222833333333334,
 'ultima': 23.222833333333334,
 'kremationen': 23.222833333333334,
 'utfärdsbönen': 23.222833333333334,
 'förrättas': 23.222833333333334,
 'antifonen': 23.222833333333334,
 'sänkes': 23.222833333333334,
 'tydningsord': 23.222833333333334,
 'fullborde': 23.222833333333334,
 'mullpåkastning': 23.222833333333334,
 'dödsfallet': 23.22283333333333,
 'kyrkogården': 23.22283333333333,
 'skal': 23.22283333333333,
 'likpredikan': 23.22283333333333,
 'kistan': 22.44873888888889,
 'hn': 22.00057894736842,
 'begravningen': 21.856784313725488,
 'begravningens': 21.77140625,
 'vigvatten': 21.77140625,
 'riten': 21.6212586206896

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is that probability that a word is discarded. Assign the subsampled data to `train_words`.

In [9]:
from collections import Counter
import random

threshold = 1e-5
word_counts = Counter(int_words)
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]